---

Created for [Pricing and Hedging Derivative Securities: Theory and Methods](https://book.derivative-securities.org/)

Authored by
- Kerry Back, Rice University
- Hong Liu, Washington University in St. Louis
- Mark Loewenstein, University of Maryland
 
---

<a target="_blank" href="https://colab.research.google.com/github/math-finance-book/book-code/blob/main/06_Deltas.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [ ]:
import numpy as np
# from bsfunctions import *
import matplotlib.pyplot as plt
import time
from math import pow, exp, sqrt
from scipy import stats
# incs = np.genfromtxt('incs.csv',delimiter=",",skip_header=1)
def blackscholes(S0, K, r, q, sig, T, call = True):
    '''Calculate option price using B-S formula.

    Args:
        S0 (num): initial price of underlying asset.
        K (num): strick price.
        r (num): risk free rate.
        q (num): dividend yield
        sig (num): Black-Scholes volatility.
        T (num): maturity.
        call (bool): True returns call price, False returns put price.

    Returns:
        num
    '''
    d1 = (np.log(S0/K) + (r -q + sig**2/2) * T)/(sig*np.sqrt(T))
    d2 = d1 - sig*np.sqrt(T)
#     norm = sp.stats.norm
    norm = stats.norm
    if call:
        return np.exp(-q*T)*S0 * norm.cdf(d1,0,1) - K * np.exp(-r * T) * norm.cdf(d2,0, 1)
    else:
        return np.exp(-q*T)*S0 * -norm.cdf(-d1,0,1) + K * np.exp(-r * T) * norm.cdf(-d2,0, 1)

def blackscholes_delta(S0, K, r, q, sig, T, call = True):
    '''Calculate option price using B-S formula.

    Args:
        S0 (num): initial price of underlying asset.
        K (num): strick price.
        r (num): risk free rate.
        q (num): dividend yield
        sig (num): Black-Scholes volatility.
        T (num): maturity.
        call (bool): True returns call price, False returns put price.

    Returns:
        num
    '''
    d1 = (np.log(S0/K) + (r -q + sig**2/2) * T)/(sig*np.sqrt(T))
    d2 = d1 - sig*np.sqrt(T)
#     norm = sp.stats.norm
    norm = stats.norm
    if type(call) == bool:
        if call:
            return np.exp(-q*T)*norm.cdf(d1,0,1)
        else:
            return np.exp(-q*T)*norm.cdf(-d1,0,1)
    else:
        print("Not a valid value for call")

# parameters
# number of paths
# n = incs.shape[1]
n = 100000
# number of divisions
# m = incs.shape[0]
m = 100
# interest rate
r = .1
# dividend yield
q=0.0
# true drift
mu = .15
# volatility
sig = .2
# Initial Stock Price
S0 = 42
# Strike Price
K = 42
# Maturity
T = 0.5


# seed for random generator
seed= 1234
# define a random generator
rg = np.random.RandomState(seed)
# initialize


# generate normal random vairables
dt= T/m
vol=sig*np.sqrt(dt)
incs = rg.normal(0,vol,[m,n])


tline = np.linspace(0,T,m+1)


St = np.zeros((m+1,n))
#St1 = np.zeros((m+1,n))

V_vec = np.zeros((m+1,n))

delta = np.zeros((m,n))

put= blackscholes(S0,K,r, q, sig,T,call=False)

incs_cumsum =  np.concatenate((np.zeros((1,n)),incs),axis=0).cumsum(axis=0)
V_vec = np.zeros((m+1,n))
t_mat =  np.repeat(tline.reshape((m+1,1)), n, axis=1)
drift_cumsum = (mu -q -0.5*sig**2) * t_mat

St = S0 * np.exp(incs_cumsum + drift_cumsum)

delta = blackscholes_delta(St[:-1,:],K,r, q, sig,T-t_mat[:-1,:])

V_vec[0,:] = S0 + put

for i in range(1,m+1):
    V_vec[i,:] = V_vec[i-1,:] + (np.exp(r*dt)-1) * (V_vec[i-1,:] - delta[i-1,:] * St[i-1,:])+ delta[i-1,:] * (St[i,:]-St[i-1,:])

# Uses actual simulated changes in riskfree and stock price not the dt and dB approximations 
# plot ST versus VT
plt.scatter(St[m,:],V_vec[m,:])
plt.xlabel('Stock Price at Maturity')
plt.ylabel('Value of Portfolio Insurance')
plt.show()

In [ ]:
import numpy as np
from scipy.stats import norm
import scipy.optimize as optimize

def simulated_delta_hedge_profit(S0, K, r, sigma, q, T, mu, M, N, pct):
    """
    Inputs:
    S0 = initial stock price
    K = strike price
    r = risk-free rate
    sigma = volatility
    q = dividend yield
    T = time to maturity
    mu = expected rate of return
    N = number of time periods
    M = number of simulations
    pct = percentile to be returned
    """
    dt = T / N
    SigSqrdt = sigma * np.sqrt(dt)
    drift = (mu - q - 0.5 * sigma ** 2) * dt
    Comp = np.exp(r * dt)
    Div = np.exp(q * dt) - 1
    LogS0 = np.log(S0)
   # Call0 = black_scholes_call(S0, K, r, sigma, q, T)
    Call0 = blackscholes(S0,K,r, q, sigma,T)
    Delta0 = blackscholes_delta(S0,K,r, q, sigma,T)
   # Delta0 = black_scholes_call_delta(S0, K, r, sigma, q, T)
    Cash0 = Call0 - Delta0 * S0
    Profit = np.zeros(M)

    for i in range(M):
        LogS = LogS0
        Cash = Cash0
        S = S0
        Delta = Delta0

        for j in range(1, N):
            LogS += drift + SigSqrdt * np.random.randn()
            NewS = np.exp(LogS)
            NewDelta = blackscholes_delta(NewS, K, r, q, sigma, T - j * dt)
            Cash = Comp * Cash + Delta * S * Div - (NewDelta - Delta) * NewS
            S = NewS
            Delta = NewDelta

        LogS += drift + SigSqrdt * np.random.randn()
        NewS = np.exp(LogS)
        HedgeValue = Comp * Cash + Delta * S * Div + Delta * NewS
        Profit[i] = HedgeValue - max(NewS - K, 0)

    return np.percentile(Profit, pct * 100)

# Example usage (you can replace these with input values)
S = 100  # Initial stock price
K = 100  # Strike price
r = 0.05  # Risk-free rate
sigma = 0.2  # Volatility
q = 0.02  # Dividend yield
T = 1  # Time to maturity in years
# CallPrice = 10  # Call price for implied volatility calculation

# Simulate delta hedging profit
S0 = 100  # Initial stock price
mu = 0.1  # Expected rate of return
M = 1000  # Number of simulations
N = 252  # Number of time periods
pct = 0.95  # Percentile to be returned

delta_hedge_profit = simulated_delta_hedge_profit(S0, K, r, sigma, q, T, mu, M, N, pct)
print(f"Delta Hedge Profit (95th percentile): {delta_hedge_profit}")